In [ ]:
!pip install boto3 pandas

In [ ]:
## Imports to load from S3 bucket
import os
import boto3

In [ ]:
## Load all pipeline parameters
key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')
model_name= os.environ.get("model_name")
bucket_name= os.environ.get("bucket")

In [ ]:
if(key_id is None or secret_key is None or bucket_name is None):
    raise TypeError("An env variables is not set")

In [ ]:
full_path= f"{bucket_name}/datasets/labeled/token-classification/{model_name}"
print(f"\n Getting from S3 bucket: {bucket_name}")
print(f"\nIn path: {full_path}")

In [ ]:
## Load s3 session
session = boto3.session.Session()
s3_client = session.client('s3',
                        region_name='nyc3',
                        endpoint_url='https://nyc3.digitaloceanspaces.com',
                        aws_access_key_id=key_id,
                        aws_secret_access_key=secret_key)

In [ ]:
paginator=s3_client.get_paginator('list_objects_v2')
pages=paginator.paginate(Bucket=bucket_name,Prefix=f"datasets/labeled/text-classification/{model_name}")

In [ ]:
import pandas as pd
def assignSeq2Df(seq):
    task=seq.get("task")
    key=list(task.get("data").keys())[0]
    input_sentence=task.get("data")[key]
    sentenceByToken=input_sentence.split(" ")
    idFromToken=[]
    for token in sentenceByToken:
        idFromToken.append(0)
    labels=seq.get("result")
    if(len(results) > 0):
        for label in labels:
            i=0
            for token in sentenceByToken:
                if(label.get("value").get("text") == token):
                    idFromToken[i]=(label.get("value").get("labels")[0])
                i+=1
    return {
    "sentenceByToken":sentenceByToken,
    "idFromToken":idFromToken
    }

In [ ]:
import json
import pandas as pd
from tqdm.notebook import tqdm
paginator=s3_client.get_paginator('list_objects_v2')
pages=paginator.paginate(Bucket=bucket_name,Prefix=f"datasets/labeled/token-classification/{model_name}")
tags=[
 "O"
]
tags2id = {
"0":0}
df=[]
for page in pages:
    page_objects = page.get('Contents', [])
    i=0
    j=1
    for obj in page_objects:
        data = s3_client.get_object(Bucket=bucket_name, Key=obj.get('Key'))
        content =(data['Body'].read()).decode("utf-8")
        if(i>0 and type(content) is str):
            annotation=json.loads(content)
            results=annotation.get("result")
            if(len(results) <= 0):
                continue
            for result in results:
                label=(result.get("value")).get("labels")[0]
                if(label not in tags2id and label not  in tags):
                    tags2id={**tags2id,label:j}
                    tags.append(label)
                    j+=1
            seq = assignSeq2Df(annotation)
            df.append(
                {
                    "_id" : i,
                "tokens":seq.get("sentenceByToken"),
                "ner_tags":seq.get("idFromToken")
                }
            )
        i+=1
df=pd.DataFrame(df)
print(f"Finished processing a total amount of {len(df.index)} annotations")

In [ ]:
### Save tags & tags2id for training metadata
with open('tags2id.json', 'w') as fp:
    json.dump(tags2id, fp)
    print("Saved tags2id.json")

with open('tags.json', 'w') as fp:
    json.dump({
    "tags":tags}, fp)
    print("Saved tags.json")

In [ ]:
### Save the dataframe to be loaded inside tran notebook

df.to_csv(path_or_buf='./dataset.csv',index=False)
print(f"\nDataframe creation completed")